In [1]:
import pandas as pd
import numpy as np
import string, os

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras.utils as ku
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
filename = 'data/wonderland.txt'
with open(filename, 'r') as f:
    raw_text = f.read().lower()

In [3]:
sentences = sent_tokenize(raw_text)

Example sentences

In [4]:
for i, s in enumerate(sentences[:2]):
    print(''.join(['=']*30 + ['{}th sentence'.format(i)] + ['=']*30))
    print(s)
print(''.join(['=']*34 + ['End'] + ['=']*35))

==============================0th sentence==============================
alice’s adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0




chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, ‘and what is the use of a book,’ thought alice ‘without pictures or
conversations?’

so she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a white rabbit with pink eyes ran
close by her.
==============================1th sentence==============================
there was nothing so very remarkable in that; nor did alice think it so
very much out of the way to hear the rabbit say to itself, ‘oh dear

In [5]:
tokenizer = Tokenizer()

Convert sentences to ngrams for prediction.

In [6]:
def tokenize_sentences(sentences, tokenizer):
    ## tokenization
    tokenizer.fit_on_texts(sentences)
    nwords = len(tokenizer.word_index) + 1
    
    # sequences of tokens 
    n_gram_sequences = []
    for line in sentences:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            n_gram_sequences.append(n_gram_sequence)
    return n_gram_sequences, nwords

n_gram_sequences, nwords = tokenize_sentences(sentences, tokenizer)
n_gram_sequences[:5]

[[318, 494],
 [318, 494, 11],
 [318, 494, 11, 889],
 [318, 494, 11, 889, 1595],
 [318, 494, 11, 889, 1595, 1596]]

Pre-pad each sentence with zeros.

In [7]:
def pad_sentences(n_gram_sequences, nwords):
    max_lenghth = max(len(x) for x in n_gram_sequences)
    padded_sequences = np.array(pad_sequences(n_gram_sequences, maxlen=max_lenghth, padding='pre'))
    
    input_sequences, labels = padded_sequences[:, :-1], padded_sequences[:, -1]
    labels = ku.to_categorical(labels, num_classes=nwords)
    return input_sequences, labels, max_lenghth

seq_X, seq_y, max_sequence_len = pad_sentences(n_gram_sequences, nwords)

In [8]:
max_sequence_len

277

Example X and y

In [9]:
for s in n_gram_sequences[:3]:
    print('text input:', tokenizer.sequences_to_texts([s[:-1]]), ' label:', tokenizer.sequences_to_texts([[s[-1]]]))
    
for i, s in enumerate(seq_X[:3]):
    print('text input:', s[s.nonzero()], ' label:', seq_y[i].nonzero()[0][0])

text input: ['alice’s']  label: ['adventures']
text input: ['alice’s adventures']  label: ['in']
text input: ['alice’s adventures in']  label: ['wonderland']
text input: [318]  label: 494
text input: [318 494]  label: 11
text input: [318 494  11]  label: 889


In [10]:
def make_model(max_len, nwords, small=True):
    input_len = max_len - 1
    model = Sequential()
    # add embedding layer
    model.add(Embedding(nwords, 10, input_length=input_len))
    if small:
        model.add(LSTM(256))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(256, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(256))
        model.add(Dropout(0.2))
    model.add(Dense(nwords, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.summary()
    return model

In [12]:
small_model = make_model(max_sequence_len, nwords)
filepath="build/word-based-weights-{epoch:02d}-{loss:.4f}-simple-small.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
small_model.fit(seq_X, seq_y, epochs=50, batch_size=64, callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 276, 10)           29150     
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               273408    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2915)              749155    
Total params: 1,051,713
Trainable params: 1,051,713
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
26853/26853 [==============================] - 149s 6ms/step - loss: 6.2553

Epoch 00001: loss improved from inf to 6.25534, saving model to build/word-based-weights-01-6.2553-simple-small.hdf5
Epoch 2/50
26853/26853 [==============================] - 1

In [13]:
big_model = make_model(max_sequence_len, nwords, small=False)
filepath="build/word-based-weights-{epoch:02d}-{loss:.4f}-simple-big.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
big_model.fit(seq_X, seq_y, epochs=50, batch_size=64, callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 276, 10)           29150     
_________________________________________________________________
lstm_2 (LSTM)                (None, 276, 256)          273408    
_________________________________________________________________
dropout_2 (Dropout)          (None, 276, 256)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2915)              749155    
Total params: 1,577,025
Trainable params: 1,577,025
Non-trainable params: 0
_________________________________________________________________


### Train on news data

In [23]:
filename = 'data/nyt-snippet-subsets.txt'
with open(filename, 'r') as f:
    raw_text = f.read().lower()

In [24]:
sentences = sent_tokenize(raw_text)

for i, s in enumerate(sentences[:2]):
    print(''.join(['=']*30 + ['{}th sentence'.format(i)] + ['=']*30))
    print(s)
print(''.join(['=']*34 + ['End'] + ['=']*35))

==============================0th sentence==============================
sleep deprivation is linked to behavioral and mental health problems and car accident risk, experts say, and starting school later could help.
==============================1th sentence==============================
the musical — a love story set during the vietnam war — ignited a fierce debate over the casting of a white actor in a eurasian role.
==================================End===================================


In [25]:
tokenizer = Tokenizer()

In [26]:
n_gram_sequences, nwords = tokenize_sentences(sentences, tokenizer)
n_gram_sequences[:5]

[[2421, 2422],
 [2421, 2422, 8],
 [2421, 2422, 8, 1426],
 [2421, 2422, 8, 1426, 3],
 [2421, 2422, 8, 1426, 3, 2423]]

In [27]:
seq_X, seq_y, max_sequence_len = pad_sentences(n_gram_sequences, nwords)

for s in n_gram_sequences[:3]:
    print('text input:', tokenizer.sequences_to_texts([s[:-1]]), ' label:', tokenizer.sequences_to_texts([[s[-1]]]))
    
for i, s in enumerate(seq_X[:3]):
    print('text input:', s[s.nonzero()], ' label:', seq_y[i].nonzero()[0][0])

text input: ['sleep']  label: ['deprivation']
text input: ['sleep deprivation']  label: ['is']
text input: ['sleep deprivation is']  label: ['linked']
text input: [2421]  label: 2422
text input: [2421 2422]  label: 8
text input: [2421 2422    8]  label: 1426


In [16]:
small_model = make_model(max_sequence_len, nwords)
filepath="build/word-based-weights-{epoch:02d}-{loss:.4f}-complex-small.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
small_model.fit(seq_X, seq_y, epochs=50, batch_size=64, callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 10)            63300     
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               273408    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6330)              1626810   
Total params: 1,963,518
Trainable params: 1,963,518
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
22620/22620 [==============================] - 41s 2ms/step - loss: 7.5416

Epoch 00001: loss improved from inf to 7.54164, saving model to build/word-based-weights-01-7.5416-complex-small.hdf5
Epoch 2/50
22620/22620 [==============================] - 3

In [17]:
big_model = make_model(max_sequence_len, nwords, small=False)
filepath="build/word-based-weights-{epoch:02d}-{loss:.4f}-complex-big.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
big_model.fit(seq_X, seq_y, epochs=50, batch_size=64, callbacks=callbacks_list)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 64, 10)            63300     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64, 256)           273408    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6330)              1626810   
Total params: 2,488,830
Trainable params: 2,488,830
Non-trainable params: 0
_________________________________________________________________


### Check point

In [2]:
def tokenize_sentences(sentences, tokenizer):
    ## tokenization
    tokenizer.fit_on_texts(sentences)
    nwords = len(tokenizer.word_index) + 1
    
    # sequences of tokens 
    n_gram_sequences = []
    for line in sentences:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            n_gram_sequences.append(n_gram_sequence)
    return n_gram_sequences, nwords

def pad_sentences(n_gram_sequences, nwords):
    max_lenghth = max(len(x) for x in n_gram_sequences)
    padded_sequences = np.array(pad_sequences(n_gram_sequences,
                                              maxlen=max_lenghth,
                                              padding='pre'))
    
    input_sequences, labels = padded_sequences[:, :-1], padded_sequences[:, -1]
    labels = ku.to_categorical(labels, num_classes=nwords)
    return input_sequences, labels, max_lenghth

def make_model(max_len, nwords, small=True):
    input_len = max_len - 1
    model = Sequential()
    # add embedding layer
    model.add(Embedding(nwords, 10, input_length=input_len))
    if small:
        model.add(LSTM(256))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(256, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(256))
        model.add(Dropout(0.2))
    model.add(Dense(nwords, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.summary()
    return model

Load models -- models trained on simple texts

In [3]:
filename = 'data/wonderland.txt'
with open(filename, 'r') as f:
    raw_text = f.read().lower()
sentences = sent_tokenize(raw_text)
tokenizer_simple = Tokenizer()
n_gram_sequences, nwords = tokenize_sentences(sentences, tokenizer_simple)
seq_X, seq_y, max_len_simple = pad_sentences(n_gram_sequences, nwords)

simple_small_model = make_model(max_len_simple, nwords)
filename = "build/word-based-weights-50-1.1499-simple-small.hdf5"
simple_small_model.load_weights(filename)
simple_small_model.compile(loss='categorical_crossentropy', optimizer='adam')

simple_big_model = make_model(max_len_simple, nwords, small=False)
filename = "build/word-based-weights-34-6.2071-simple-big.hdf5"
simple_big_model.load_weights(filename)
simple_big_model.compile(loss='categorical_crossentropy', optimizer='adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 276, 10)           29150     
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               273408    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2915)              749155    
Total params: 1,051,713
Trainable params: 1,051,713
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 276, 10)           29150     
_________________________________________________________________


Models trained on complex texts

In [4]:
filename = 'data/nyt-snippet-subsets.txt'
with open(filename, 'r') as f:
    raw_text = f.read().lower()
sentences = sent_tokenize(raw_text)
tokenizer_complex = Tokenizer()
n_gram_sequences, nwords = tokenize_sentences(sentences, tokenizer_complex)
seq_X, seq_y, max_len_complex = pad_sentences(n_gram_sequences, nwords)

complex_small_model = make_model(max_len_complex, nwords)
filename = "build/word-based-weights-50-1.1208-complex-small.hdf5"
complex_small_model.load_weights(filename)
complex_small_model.compile(loss='categorical_crossentropy', optimizer='adam')

complex_big_model = make_model(max_len_complex, nwords, False)
filename = "build/word-based-weights-04-7.6220-complex-big.hdf5"
complex_big_model.load_weights(filename)
complex_big_model.compile(loss='categorical_crossentropy', optimizer='adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 64, 10)            63300     
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               273408    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6330)              1626810   
Total params: 1,963,518
Trainable params: 1,963,518
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 64, 10)            63300     
_________________________________________________________________


Word stats

In [6]:
print('AAiW')
print('Unique words: {0} the total number of words: {1}'.format(len(tokenizer_simple.word_counts), sum(x for x in tokenizer_simple.word_counts.values())))
print('NYT')
'Unique words: {0} the total number of words: {1}'.format(len(tokenizer_complex.word_counts), sum(x for x in tokenizer_complex.word_counts.values()))

AAiW
Unique words: 2914 the total number of words: 27827
NYT


'Unique words: 6329 the total number of words: 24049'

### Text generation
- generate texts by trained four models using the following four texts.

In [5]:
CNN = 'Occupy Boston protesters remained firmly entrenched in a downtown city square early Friday after a midnight deadline passed for them to clear out or face eviction.  Demonstrators cleared trash and some of the more than 100 tents from the area, but most stayed put. Police presence was light around Dewey Square. Superior Court Judge Frances A. McIntyre ruled Wednesday that demonstrators\' First Amendment rights do not extend to seizing and holding areas on which they sit. Authorities are \"obligated by law to preserve Dewey Square as a space open to the public,\" McIntyre added. Boston Mayor Thomas M. Menino, who in the past has appeared to tolerate his city\'s chapter of the nationwide movement, recently signaled that the park could be a safety hazard as winter weather rolls in. \"We\'re asking them to leave, according to their own will and volition,\" Menino told CNN affiliate WCVB. \"After that, we\'ll make decisions about how we\'ll clear off the site in the future.\" Protesters have been encamped in the square since late September.  On Thursday, police could be seen handing out fliers to protesters in an apparent effort to inform them that they soon had to leave.  \"It\'s like telling you to get out of your apartment by midnight,\" protester Philip O\'Connell told the affiliate station. By Thursday evening, comments on an Occupy Boston Twitter feed were focused on the impending deadline.  \"Some tents may fold, but an idea cannot be evicted,\" read one post.  \"We have occupied Dewey Square because Wall Street has occupied our government,\" read another. In October, 129 people were arrested during a demonstration in which protesters allegedly blocked traffic and refused to disperse, according to police spokesman Eddy Chrispin. They were arrested mostly for \"unlawful assembly and trespassing,\" he said.  The movement, which first sprang up in a Lower Manhattan park, seeks to highlight what it sees as corruption and growing income disparities between the nation\'s richest 1% and the rest of the country. In a move similar to McIntyre\'s ruling, a New York Supreme Court announced last month that Occupy protesters would be allowed to return to Zuccotti Park -- considered a home base for demonstrators -- but would be restricted from camping overnight.  Police in riot gear cleared them out in the early-morning hours in mid-November, a move that attorneys for the demonstrators have said was unlawful. Thousands later deluged the city\'s financial district in a show of strength echoed nationwide as part of the group\'s so-called \"mass day of action.\" Scores were arrested across the city, and several police officers were reported injured, as thousands of others gathered in places such as St. Louis, San Francisco, Denver and Milwaukee to respond to the \"day of action\" plea. Last month, police in Los Angeles and Philadelphia dismantled tents and arrested Occupy protesters who refused to leave city areas.  In Philadelphia, police arrested 52 people after scuffles broke out when authorities ordered some protesters to clear the street.  Encampments have largely remained in a handful of cities, including San Francisco; Asheville, North Carolina; Oklahoma City; and Washington, according to media reports and group websites. '
USAToday = 'Shopping carts sit outside the Wal-Mart store in Mayfield Hts., Ohio., on Nov. 14, 2011. The retailer offered few details about the investigation in a quarterly report filed Thursday with the Securities and Exchange Commission. The company said that it opened the investigation after reviewing policies, procedures and internal controls tied to its global anti-corruption program. It said that it is taking \"appropriate remedial measures.\" \"We are taking a deep look at our policies and procedures in every country in which we operate,\" said Wal-Mart spokesman Dave Tovar in an email. \"As a result of information obtained during that review and from other sources, we have begun an internal investigation related to compliance with the\" Foreign Corrupt Practice Act. All companies doing business overseas must comply with the U.S. Foreign Corrupt Practices Act, which broadly deals with bribery and accounting rules. Wal-Mart Stores (WMT) has hired outside lawyers and other advisers and has notified the U.S. Department of Justice and Securities and Exchange Commission. \"Our investigation is currently focused on discrete incidents in specific areas,\" Tovar said. \"We intend to keep federal authorities apprised of what we learn.\" The issue will not have a material impact on business, the company said. The world\'s largest retailer — which operates in 28 countries, including China, Mexico and Japan — brought in $419 billion in revenue last year. Company shares fell 31 cents or 0.5% to $58.29 on Friday. '
TheGreenMile = 'This happened in 1932, when the state penitentiary was still at Cold Mountain. And the electric chair was there, too, of course.   The inmates made jokes about the chair the way people always make jokes about things that frighten them but can\'t be gotten away from. They called it Old Sparky, or the Big Juicy. They made cracks about the Power bill, and how Warden Moores would cook his Thanksgiving dinner that fall, with his wife, Melinda, too sick to cook.   But for the ones who actually had to sit down in that chair, the humor went out of the situation in a hurry I presided over seventy-eight executions during my time at Cold Mountain (that\'s one figure I\'ve never been confused about; I\'ll remember it on my deathbed), and I think that, for most of those men, the truth of what was happening to them finally hit all the way home when their ankles were being damped to the stout oak of \"Old Sparky\'s\" legs. The realization came then (you would see it rising in their eyes, a kind of cold dismay) that their, own legs had finished their careers. The blood still ran in them, the muscles were still strong, but they were finished, all the same; they were never going to walk another country mile or dance with a girl at a barn-raising. Old Sparky\'s clients came to a knowledge of their deaths from the ankles up. There was a black silk bag that went over their heads after they had finished their rambling and mostly disjointed last remarks. It was supposed to be for them, but I always thought it was really for us, to keep us from seeing the awful tide of dismay in their eyes as they realized they were going to die with their knees bent.   There was no death row at Cold Mountain, only E Block, set apart from the other four and about a quarter their size, brick instead of wood, with a horrible bare metal roof that glared in the summer sun like a delirious eyeball. Six cells inside, three on each side of a wide center aisle, each almost twice as big as the cells in the other four blocks. Singles, too. Great accommodations for a prison (especially in the thirties), but the inmates would have traded for cells in any of the other four. Believe me, they would have traded.   There was never a time during my years as block superintendent when all six cells were occupied at one time -- thank God for small favors. Four was the most, mixed black and white (at Cold Mountain, there was no segregation among the walking dead), and that was a little piece of hell. One was a woman, Beverly McCall. She was black as the ace of spades and as beautiful as the sin you never had nerve enough to commit. She put up with six years of her husband beating her, but wouldn\'t put up with his creeping around for a single day. On the evening after she found out he was cheating, she stood waiting for the unfortunate Lester McCall, known to his pals (and, presumably, to his extremely short-term mistress) as Cutter, at the top of the stairs leading to the apartment over his barber shop. She waited until he got his overcoat half off, then dropped his cheating guts onto his tu-tone shoes. Used one of Cutter\'s own razors to do it. Two nights before she was due to sit in Old Sparky, she called me to her cell and said she had been visited by her African spirit-father in a dream. He told her to discard her slave-name and to die under her free name, Matuomi. That was her request, that her deathwarrant should be read under the name of Beverly Matuomi. I guess her spirit-father didn\'t give her any first name, or one she could make out, anyhow. I said yes, okay, fine. One thing those years serving as the bull-goose screw taught me was never to refuse the condemned unless I absolutely had to. In the case of Beverly Matuomi, it made no difference, anyway. The governor called the next day around three in the afternoon, commuting her sentence to life in the Grassy Valley Penal Facility for Women -- all penal and no penis, we used to say back then. I was glad to see Bev\'s round ass going left instead of right when she got to the duty desk, let me tell you. Thirty-five years or so later -- had to be at least thirty-five -- I saw that name on the obituary page of the paper, under a picture of a skinny-faced black lady with a cloud of white hair and glasses with rhinestones at the comers. It was Beverly. She\'d spent the last ten years of her life a free woman, the obituary said, and had rescued the small-town library of Raines Falls pretty much single-handed. She had also taught Sunday school and had been much loved in that little backwater. LIBRARIAN DIES OF HEART FAILURE, the headline said, and below that, in smaller type, almost as an afterthought: Served Over Two Decades in Prison for Murder. Only the eyes, wide and blazing behind the glasses with the rhinestones at the comers, were the same. They were the eyes of a woman who even at seventy-whatever would not hesitate to pluck a safety razor from its blue jar of disinfectant, if the urge seemed pressing. You know murderers, even if they finish up as old lady librarians in dozey little towns. At least you do if you\'ve spent as much time minding murderers as I did. There was only one time I ever had a question about the nature of my job. That, I reckon, is why I\'m writing this.   The wide corridor up the center of E Block was floored with linoleum the color of tired old limes, and so what was called the Last Mile at other prisons was called the Green Mile at Cold Mountain. It ran, I guess, sixty long paces from south to north, bottom to top. At the bottom was the restraint room. At the top end was a T-junction. A left turn meant life -- if you called what went on in the sunbaked exercise yard life, and many did; many lived it for years, with no apparent ill effects. Thieves and arsonists and sex criminals, all talking their talk and walking their walk and making their little deals.   A right turn, though -- that was different. First you went into my office (where the carpet was also green, a thing I kept meaning to change and not getting around to), and crossed in front of my desk, which was flanked by the American flag on the left and the state flag on the right. On the far side were two doors. One led into the small W.C. that I and the E Block guards (sometimes even Warden Moores) used; the other opened on a kind of storage shed. This was where you ended up when you walked the Green Mile.   It was a small door -- I had to duck my head when I went through, and John Coffey actually had to sit and scoot. You came out on a little landing, then went down three cement steps to a board floor. It was a miserable room without heat and with a metal roof, just like the one on the block to which it was an adjunct. It was cold enough in there to see your breath during the winter, and stifling in the summer. At the execution of Elmer Manfred -- in July or August of \'30, that one was, I believe -- we had nine witnesses pass out.   On the left side of the storage shed -- again -- there was life. Tools (all locked down in frames crisscrossed with chains, as if they were carbine rifles instead of spades and pickaxes), dry goods, sacks of seeds for spring planting in the prison gardens, boxes of toilet paper, pallets cross-loaded with blanks for the prison plate-shop...even bags of lime for marking out the baseball diamond and the football gridiron -- the cons played in what was known as The Pasture, and fall afternoons were greatly looked forward to at Cold Mountain.   On the right -- once again -- death. Old Sparky his ownself, sitting up on a plank platform at the southeast comer of the storeroom, stout oak legs, broad oak arms that had absorbed the terrorized sweat of scores of men in the last few minutes of their lives, and the metal cap, usually hung jauntily on the back of the chair, like some robot kid\'s beanie in a Buck Rogers comic-strip. A cord ran from it and through a gasket-circled hole in the cinderblock wall behind the chair. Off to one side was a galvanized tin bucket. If you looked inside it, you would see a circle of sponge, cut just right to fit the metal cap. Before executions, it was soaked in brine to better conduct the charge of direct-current electricity that ran through the wire, through the sponge, and into the condemned man\'s brain.'
HarryPotter = 'October arrived, spreading a damp chill over the grounds and into the castle. Madam Pomfrey, the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. Ginny Weasley, who had been looking pale, was bullied into taking some by Percy. The steam pouring from under her vivid hair gave the impression that her whole head was on fire.  Raindrops the size of bullets thundered on the castle windows for days on end; the lake rose, the flower beds turned into muddy streams, and Hagrid \'s pumpkins swelled to the size of garden sheds. Oliver Wood \'s enthusiasm for regular training sessions, however, was not dampened, which was why Harry was to be found, late one stormy Saturday afternoon a few days before Halloween, returning to Gryffindor Tower, drenched to the skin and splattered with mud.  Even aside from the rain and wind it hadn \'t been a happy practice session. Fred and George, who had been spying on the Slytherin team, had seen for themselves the speed of those new Nimbus Two Thousand and Ones. They reported that the Slytherin team was no more than seven greenish blurs, shooting through the air like missiles.  As Harry squelched along the deserted corridor he came across somebody who looked just as preoccupied as he was. Nearly Headless Nick, the ghost of Gryffindor Tower, was staring morosely out of a window, muttering under his breath,  \"… don \'t fulfill their requirements… half an inch, if that… \"   \"Hello, Nick, \" said Harry.   \"Hello, hello, \" said Nearly Headless Nick, starting and looking round. He wore a dashing, plumed hat on his long curly hair, and a tunic with a ruff, which concealed the fact that his neck was almost completely severed. He was pale as smoke, and Harry could see right through him to the dark sky and torrential rain outside.   \"You look troubled, young Potter, \" said Nick, folding a transparent letter as he spoke and tucking it inside his doublet.   \"So do you, \" said Harry.   \"Ah, \" Nearly Headless Nick waved an elegant hand,  \"a matter of no importance.… It \'s not as though I really wanted to join.… Thought I \'d apply, but apparently I  \'don \'t fulfill requirements \' - \"  In spite of his airy tone, there was a look of great bitterness on his face.   \"But you would think, wouldn \'t you, \" he erupted suddenly, pulling the letter back out of his pocket,  \"that getting hit forty-five times in the neck with a blunt axe would qualify you to join the Headless Hunt? \"   \"Oh - yes, \" said Harry, who was obviously supposed to agree.   \"I mean, nobody wishes more than I do that it had all been quick and clean, and my head had come off properly, I mean, it would have saved me a great deal of pain and ridicule. However - \" Nearly Headless Nick shook his letter open and read furiously:  \" \'We can only accept huntsmen whose heads have parted company with their bodies. You will appreciate that it would be impossible otherwise for members to participate in hunt activities such as Horseback Head-Juggling and Head Polo. It is with the greatest regret, therefore, that I must inform you that you do not fulfill our requirements. With very best wishes, Sir Patrick Delaney-Podmore. \' \"  Fuming, Nearly Headless Nick stuffed the letter away.   \"Half an inch of skin and sinew holding my neck on, Harry! Most people would think that \'s good and beheaded, but oh, no, it \'s not enough for Sir Properly Decapitated-Podmore. \"  Nearly Headless Nick took several deep breaths and then said, in a far calmer tone,  \"So - what \'s bothering you? Anything I can do? \"   \"No, \" said Harry.  \"Not unless you know where we can get seven free Nimbus Two Thousand and Ones for our match against Sly - \"  The rest of Harry \'s sentence was drowned out by a high-pitched mewling from somewhere near his ankles. He looked down and found himself gazing into a pair of lamp-like yellow eyes. It was Mrs. Norris, the skeletal gray cat who was used by the caretaker, Argus Filch, as a sort of deputy in his endless battle against students.   \"You \'d better get out of here, Harry, \" said Nick quickly.  \"Filch isn \'t in a good mood - he \'s got the flu and some third years accidentally plastered frog brains all over the ceiling in dungeon five. He \'s been cleaning all morning, and if he sees you dripping mud all over the place - \"   \"Right, \" said Harry, backing away from the accusing stare of Mrs. Norris, but not quickly enough. Drawn to the spot by the mysterious power that seemed to connect him with his foul cat, Argus Filch burst suddenly through a tapestry to Harry \'s right, wheezing and looking wildly about for the rule-breaker. There was a thick tartan scarf bound around his head, and his nose was unusually purple.   \"Filth! \" he shouted, his jowls aquiver, his eyes popping alarmingly as he pointed at the muddy puddle that had dripped from Harry \'s Quidditch robes.  \"Mess and muck everywhere! I \'ve had enough of it, I tell you! Follow me, Potter! \"  So Harry waved a gloomy good-bye to Nearly Headless Nick and followed Filch back downstairs, doubling the number of muddy footprints on the floor.  Harry had never been inside Filch \'s office before; it was a place most students avoided. The room was dingy and windowless, lit by a single oil lamp dangling from the low ceiling. A faint smell of fried fish lingered about the place. Wooden filing cabinets stood around the walls; from their labels, Harry could see that they contained details of every pupil Filch had ever punished. Fred and George Weasley had an entire drawer to themselves. A highly polished collection of chains and manacles hung on the wall behind Filch \'s desk. It was common knowledge that he was always begging Dumbledore to let him suspend students by their ankles from the ceiling.  Filch grabbed a quill from a pot on his desk and began shuffling around looking for parchment.   \"Dung, \" he muttered furiously,  \"great sizzling dragon bogies… frog brains… rat intestines… I \'ve had enough of it… make an example… where \'s the form… yes… \"  He retrieved a large roll of parchment from his desk drawer and stretched it out in front of him, dipping his long black quill into the ink pot.   \"Name… Harry Potter. Crime… \"   \"It was only a bit of mud! \" said Harry.   \"It \'s only a bit of mud to you, boy, but to me it \'s an extra hour scrubbing! \" shouted Filch, a drip shivering unpleasantly at the end of his bulbous nose.  \"Crime… befouling the castle… suggested sentence… \"  Dabbing at his streaming nose, Filch squinted unpleasantly at Harry who waited with bated breath for his sentence to fall.  But as Filch lowered his quill, there was a great BANG! on the ceiling of the office, which made the oil lamp rattle.   \"PEEVES! \" Filch roared, flinging down his quill in a transport of rage.  \"I \'ll have you this time, I \'ll have you! \"  And without a backward glance at Harry, Filch ran flat-footed from the office, Mrs. Norris streaking alongside him.  Peeves was the school poltergeist, a grinning, airborne menace who lived to cause havoc and distress. Harry didn \'t much like Peeves, but couldn \'t help feeling grateful for his timing. Hopefully, whatever Peeves had done (and it sounded as though he \'d wrecked something very big this time) would distract Filch from Harry.  Thinking that he should probably wait for Filch to come back, Harry sank into a moth-eaten chair next to the desk. There was only one thing on it apart from his half-completed form: a large, glossy, purple envelope with silver lettering on the front. With a quick glance at the door to check that Filch wasn \'t on his way back, Harry picked up the envelope and read: kwikspell A Correspondence Course in Beginners \' Magic  p>Intrigued, Harry flicked the envelope open and pulled out the sheaf of parchment inside. More curly silver writing on the front page said: Feel out of step in the world of modern magic? Find yourself making excuses not to perform simple spells? Ever been taunted for your woeful wandwork? There is an answer! Kwikspell is an all-new, fail-safe, quick-result, easy-learn course. Hundreds of witches and wizards have benefited from the Kwikspell method! Madam Z. Nettles of Topsham writes:  \"I had no memory for incantations and my potions were a family joke! Now, after a Kwikspell course, I am the center of attention at parties and friends beg for the recipe of my Scintillation Solution! \" Warlock D. J. Prod of Didsbury says:  \"My wife used to sneer at my feeble charms, but one month into your fabulous Kwikspell course and I succeeded in turning her into a yak! Thank you, Kwikspell! \"    Fascinated, Harry thumbed through the rest of the envelope \'s contents. Why on earth did Filch want a Kwikspell course? Did this mean he wasn \'t a proper wizard? Harry was just reading  \"Lesson One: Holding Your Wand (Some Useful Tips) \" when shuffling footsteps outside told him Filch was coming back. Stuffing the parchment back into the envelope, Harry threw it back onto the desk just as the door opened.  Filch was looking triumphant.   \"That vanishing cabinet was extremely valuable! \" he was saying gleefully to Mrs. Norris.  \"We \'ll have Peeves out this time, my sweet - \"  His eyes fell on Harry and then darted to the Kwikspell envelope, which, Harry realized too late, was lying two feet away from where it had started.  Filch \'s pasty face went brick red. Harry braced himself for a tidal wave of fury. Filch hobbled across to his desk, snatched up the envelope, and threw it into a drawer.   \"Have you - did you read -? \" he sputtered.   \"No, \" Harry lied quickly.  Filch \'s knobbly hands were twisting together.   \"If I thought you \'d read my private - not that it \'s mine - for a friend - be that as it may - however - \"  Harry was staring at him, alarmed; Filch had never looked madder. His eyes were popping, a tic was going in one of his pouchy cheeks, and the tartan scarf didn \'t help.   \"Very well - go - and don \'t breathe a word - not that - however, if you didn \'t read - go now, I have to write up Peeves \' report - go - \"  Amazed at his luck, Harry sped out of the office, up the corridor, and back upstairs. To escape from Filch \'s office without punishment was probably some kind of school record.   \"Harry! Harry! Did it work? \"  Nearly Headless Nick came gliding out of a classroom. Behind him, Harry could see the wreckage of a large black-and-gold cabinet that appeared to have been dropped from a great height.   \"I persuaded Peeves to crash it right over Filch \'s office, \" said Nick eagerly.  \"Thought it might distract him - \"   \"Was that you? \" said Harry gratefully.  \"Yeah, it worked, I didn \'t even get detention. Thanks, Nick! \"  They set off up the corridor together. Nearly Headless Nick, Harry noticed, was still holding Sir Patrick \'s rejection letter.   \"I wish there was something I could do for you about the Headless Hunt, \" Harry said.    Nearly Headless Nick stopped in his tracks and Harry walked right through him. He wished he hadn \'t; it was like stepping through an icy shower.   \"But there is something you could do for me, \" said Nick excitedly.  \"Harry - would I be asking too much - but no, you wouldn \'t want - \"   \"What is it? \" said Harry.   \"Well, this Halloween will be my five hundredth deathday, \" said Nearly Headless Nick, drawing himself up and looking dignified.   \"Oh, \" said Harry, not sure whether he should look sorry or happy about this.  \"Right. \"   \"I \'m holding a party down in one of the roomier dungeons. Friends will be coming from all over the country. It would be such an honor if you would attend. Mr. Weasley and Miss Granger would be most welcome, too, of course - but I daresay you \'d rather go to the school feast? \" He watched Harry on tenterhooks.   \"No, \" said Harry quickly,  \"I \'ll come - \"   \"My dear boy! Harry Potter, at my deathday party! And \" - he hesitated, looking excited -  \"do you think you could possibly mention to Sir Patrick how very frightening and impressive you find me? \"   \"Of - of course, \" said Harry.  Nearly Headless Nick beamed at him.  \"A deathday party? \" said Hermione keenly when Harry had changed at last and joined her and Ron in the common room.  \"I bet there aren \'t many living people who can say they \'ve been to one of those - it \'ll be fascinating! \"   \"Why would anyone want to celebrate the day they died? \" said Ron, who was halfway through his Potions homework and grumpy.  \"Sounds dead depressing to me.… \"  Rain was still lashing the windows, which were now inky black, but inside all looked bright and cheerful. The firelight glowed over the countless squashy armchairs where people sat reading, talking, doing homework or, in the case of Fred and George Weasley, trying to find out what would happen if you fed a Filibuster firework to a salamander. Fred had  \"rescued \" the brilliant orange, fire-dwelling lizard from a Care of Magical Creatures class and it was now smouldering gently on a table surrounded by a knot of curious people.  Harry was at the point of telling Ron and Hermione about Filch and the Kwikspell course when the salamander suddenly whizzed into the air, emitting loud sparks and bangs as it whirled wildly round the room. The sight of Percy bellowing himself hoarse at Fred and George, the spectacular display of tangerine stars showering from the salamander \'s mouth, and its escape into the fire, with accompanying explosions, drove both Filch and the Kwikspell envelope from Harry \'s mind. By the time Halloween arrived, Harry was regretting his rash promise to go to the deathday party. The rest of the school was happily anticipating their Halloween feast; the Great Hall had been decorated with the usual live bats, Hagrid \'s vast pumpkins had been carved into lanterns large enough for three men to sit in, and there were rumors that Dumbledore had booked a troupe of dancing skeletons for the entertainment.   \"A promise is a promise, \" Hermione reminded Harry bossily.  \"You said you \'d go to the deathday party. \"  So at seven o \'clock, Harry, Ron, and Hermione walked straight past the doorway to the packed Great Hall, which was glittering invitingly with gold plates and candles, and directed their steps instead toward the dungeons.  The passageway leading to Nearly Headless Nick \'s party had been lined with candles, too, though the effect was far from cheerful: These were long, thin, jet-black tapers, all burning bright blue, casting a dim, ghostly light even over their own living faces. The temperature dropped with every step they took. As Harry shivered and drew his robes tightly around him, he heard what sounded like a thousand fingernails scraping an enormous blackboard.   \"Is that supposed to be music? \" Ron whispered. They turned a corner and saw Nearly Headless Nick standing at a doorway hung with black velvet drapes.   \"My dear friends, \" he said mournfully.  \"Welcome, welcome… so pleased you could come.… \"  He swept off his plumed hat and bowed them inside.  It was an incredible sight. The dungeon was full of hundreds of pearly-white, translucent people, mostly drifting around a crowded dance floor, waltzing to the dreadful, quavering sound of thirty musical saws, played by an orchestra on a raised, black-draped platform. A chandelier overhead blazed midnight-blue with a thousand more black candles. Their breath rose in a mist before them; it was like stepping into a freezer.   \"Shall we have a look around? \" Harry suggested, wanting to warm up his feet.   \"Careful not to walk through anyone, \" said Ron nervously, and they set off around the edge of the dance floor. They passed a group of gloomy nuns, a ragged man wearing chains, and the Fat Friar, a cheerful Hufflepuff ghost, who was talking to a knight with an arrow sticking out of his forehead. Harry wasn \'t surprised to see that the Bloody Baron, a gaunt, staring Slytherin ghost covered in silver bloodstains, was being given a wide berth by the other ghosts.   \"Oh, no, \" said Hermione, stopping abruptly.  \"Turn back, turn back, I don \'t want to talk to Moaning Myrtle - \"   \"Who? \" said Harry as they backtracked quickly.   \"She haunts one of the toilets in the girls \' bathroom on the first floor, \" said Hermione.   \"She haunts a toilet? \"   \"Yes. It \'s been out-of-order all year because she keeps having tantrums and flooding the place. I never went in there anyway if I could avoid it; it \'s awful trying to have a pee with her wailing at you - \"  \"Look, food! \" said Ron.  On the other side of the dungeon was a long table, also covered in black velvet. They approached it eagerly but next moment had stopped in their tracks, horrified. The smell was quite disgusting. Large, rotten fish were laid on handsome silver platters; cakes, burned charcoal-black, were heaped on salvers; there was a great maggoty haggis, a slab of cheese covered in furry green mold and, in pride of place, an enormous gray cake in the shape of a tombstone, with tar-like icing forming the words, Sir Nicholas de Mimsy-Porpington died 31st October, 1492  Harry watched, amazed, as a portly ghost approached the table, crouched low, and walked through it, his mouth held wide so that it passed through one of the stinking salmon.   \"Can you taste it if you walk though it? \" Harry asked him.   \"Almost, \" said the ghost sadly, and he drifted away.   \"I expect they \'ve let it rot to give it a stronger flavor, \" said Hermione knowledgeably, pinching her nose and leaning closer to look at the putrid haggis.   \"Can we move? I feel sick, \" said Ron.  They had barely turned around, however, when a little man swooped suddenly from under the table and came to a halt in midair before them.   \"Hello, Peeves, \" said Harry cautiously.  Unlike the ghosts around them, Peeves the Poltergeist was the very reverse of pale and transparent. He was wearing a bright orange party hat, a revolving bow tie, and a broad grin on his wide, wicked face.   \"Nibbles? \" he said sweetly, offering them a bowl of peanuts covered in fungus.   \"No thanks, \" said Hermione.   \"Heard you talking about poor Myrtle, \" said Peeves, his eyes dancing.  \"Rude you was about poor Myrtle. \" He took a deep breath and bellowed,  \"OY! MYRTLE! \"   \"Oh, no, Peeves, don \'t tell her what I said, she \'ll be really upset, \" Hermione whispered frantically.  \"I didn \'t mean it, I don \'t mind her - er, hello, Myrtle. \"  The squat ghost of a girl had glided over. She had the glummest face Harry had ever seen, half-hidden behind lank hair and thick, pearly spectacles.   \"What? \" she said sulkily.   \"How are you, Myrtle? \" said Hermione in a falsely bright voice.  \"It \'s nice to see you out of the toilet. \"  Myrtle sniffed.   \"Miss Granger was just talking about you - \" said Peeves slyly in Myrtle \'s ear.   \"Just saying - saying - how nice you look tonight, \" said Hermione, glaring at Peeves.  Myrtle eyed Hermione suspiciously.   \"You \'re making fun of me, \" she said, silver tears welling rapidly in her small, see-through eyes.   \"No - honestly - didn \'t I just say how nice Myrtle \'s looking? \" said Hermione, nudging Harry and Ron painfully in the ribs.   \"Oh, yeah - \"   \"She did - \"   \"Don \'t lie to me, \" Myrtle gasped, tears now flooding down her face, while Peeves chuckled happily over her shoulder.  \"D \'you think I don \'t know what people call me behind my back? Fat Myrtle! Ugly Myrtle! Miserable, moaning, moping Myrtle! \"   \"You \'ve forgotten pimply, \" Peeves hissed in her ear.  Moaning Myrtle burst into anguished sobs and fled from the dungeon. Peeves shot after her, pelting her with moldy peanuts, yelling,  \"Pimply! Pimply! \"   \"Oh, dear, \" said Hermione sadly.  Nearly Headless Nick now drifted toward them through the crowd.   \"Enjoying yourselves? \"   \"Oh, yes, \" they lied.   \"Not a bad turnout, \" said Nearly Headless Nick proudly.  \"The Wailing Widow came all the way up from Kent.… It \'s nearly time for my speech, I \'d better go and warn the orchestra.… \"  The orchestra, however, stopped playing at that very moment. They, and everyone else in the dungeon, fell silent, looking around in excitement, as a hunting horn sounded.   \"Oh, here we go, \" said Nearly Headless Nick bitterly.  Through the dungeon wall burst a dozen ghost horses, each ridden by a headless horseman. The assembly clapped wildly; Harry started to clap, too, but stopped quickly at the sight of Nick \'s face.  The horses galloped into the middle of the dance floor and halted, rearing and plunging. At the front of the pack was a large ghost who held his bearded head under his arm, from which position he was blowing the horn. The ghost leapt down, lifted his head high in the air so he could see over the crowd (everyone laughed), and strode over to Nearly Headless Nick, squashing his head back onto his neck.   \"Nick! \" he roared.  \"How are you? Head still hanging in there? \"  He gave a hearty guffaw and clapped Nearly Headless Nick on the shoulder.   \"Welcome, Patrick, \" said Nick stiffly.   \"Live  \'uns! \" said Sir Patrick, spotting Harry, Ron, and Hermione and giving a huge, fake jump of astonishment, so that his head fell off again (the crowd howled with laughter).   \"Very amusing, \" said Nearly Headless Nick darkly.   \"Don \'t mind Nick! \" shouted Sir Patrick \'s head from the floor.  \"Still upset we won \'t let him join the Hunt! But I mean to say - look at the fellow - \"   \"I think, \" said Harry hurriedly, at a meaningful look from Nick,  \"Nick \'s very - frightening and - er - \"   \"Ha! \" yelled Sir Patrick \'s head.  \"Bet he asked you to say that! \"   \"If I could have everyone \'s attention, it \'s time for my speech! \" said Nearly Headless Nick loudly, striding toward the podium and climbing into an icy blue spotlight.   \"My late lamented lords, ladies, and gentlemen, it is my great sorrow… \"  But nobody heard much more. Sir Patrick and the rest of the Headless Hunt had just started a game of Head Hockey and the crowd were turning to watch. Nearly Headless Nick tried vainly to recapture his audience, but gave up as Sir Patrick \'s head went sailing past him to loud cheers.  Harry was very cold by now, not to mention hungry.   \"I can \'t stand much more of this, \" Ron muttered, his teeth chattering, as the orchestra ground back into action and the ghosts swept back onto the dance floor.   \"Let \'s go, \" Harry agreed.  They backed toward the door, nodding and beaming at anyone who looked at them, and a minute later were hurrying back up the passageway full of black candles.   \"Pudding might not be finished yet, \" said Ron hopefully, leading the way toward the steps to the entrance hall.  And then Harry heard it.   \"… rip… tear… kill… \"  It was the same voice, the same cold, murderous voice he had heard in Lockhart \'s office.  He stumbled to a halt, clutching at the stone wall, listening with all his might, looking around, squinting up and down the dimly lit passageway.   \"Harry, what \'re you -? \"   \"It \'s that voice again - shut up a minute - \"   \"… soo hungry… for so long… \"   \"Listen! \" said Harry urgently, and Ron and Hermione froze, watching him.   \"… kill… time to kill… \"  The voice was growing fainter. Harry was sure it was moving away - moving upward. A mixture of fear and excitement gripped him as he stared at the dark ceiling; how could it be moving upward? Was it a phantom, to whom stone ceilings didn \'t matter?   \"This way, \" he shouted, and he began to run, up the stairs, into the entrance hall. It was no good hoping to hear anything here, the babble of talk from the Halloween feast was echoing out of the Great Hall. Harry sprinted up the marble staircase to the first floor, Ron and Hermione clattering behind him.   \"Harry, what \'re we - \"   \"SHH! \"  Harry strained his ears. Distantly, from the floor above, and growing fainter still, he heard the voice:  \"… I smell blood.… I SMELL BLOOD! \"  His stomach lurched -   \"It \'s going to kill someone! \" he shouted, and ignoring Ron \'s and Hermione \'s bewildered faces, he ran up the next flight of steps three at a time, trying to listen over his own pounding footsteps -  Harry hurtled around the whole of the second floor, Ron and Hermione panting behind him, not stopping until they turned a corner into the last, deserted passage.   \"Harry, what was that all about? \" said Ron, wiping sweat off his face.  \"I couldn \'t hear anything.… \"  But Hermione gave a sudden gasp, pointing down the corridor.   \"Look! \"  Something was shining on the wall ahead. They approached slowly, squinting through the darkness. Foot-high words had been daubed on the wall between two windows, shimmering in the light cast by the flaming torches. the chamber of secrets has been opened. enemies of the heir, beware.   \"What \'s that thing - hanging underneath? \" said Ron, a slight quiver in his voice.  As they edged nearer, Harry almost slipped - there was a large puddle of water on the floor; Ron and Hermione grabbed him, and they inched toward the message, eyes fixed on a dark shadow beneath it. All three of them realized what it was at once, and leapt backward with a splash.  Mrs. Norris, the caretaker \'s cat, was hanging by her tail from the torch bracket. She was stiff as a board, her eyes wide and staring.  For a few seconds, they didn \'t move. Then Ron said,  \"Let \'s get out of here. \"   \"Shouldn \'t we try and help - \" Harry began awkwardly.   \"Trust me, \" said Ron.  \"We don \'t want to be found here. \"  But it was too late. A rumble, as though of distant thunder, told them that the feast had just ended. From either end of the corridor where they stood came the sound of hundreds of feet climbing the stairs, and the loud, happy talk of well-fed people; next moment, students were crashing into the passage from both ends.  The chatter, the bustle, the noise died suddenly as the people in front spotted the hanging cat. Harry, Ron, and Hermione stood alone, in the middle of the corridor, as silence fell among the mass of students pressing forward to see the grisly sight.  Then someone shouted through the quiet.   \"Enemies of the Heir, beware! You \'ll be next, Mudbloods! \"  It was Draco Malfoy. He had pushed to the front of the crowd, his cold eyes alive, his usually bloodless face flushed, as he grinned at the sight of the hanging, immobile cat. '

In [6]:
source_name = ['cnn', 'usa-today', 'the-green-mile', 'harry-potter']

In [7]:
def generate_text(input_words, n_output_words, model, tokenizer, max_len=277):
    index_word = dict((i, w) for w, i in tokenizer.word_index.items())
    for _ in range(n_output_words):
        input_seq = tokenizer.texts_to_sequences([input_words])[0]
        input_seq = input_seq[:(max_len-1)]
        padded_seq = pad_sequences([input_seq], maxlen=max_len-1, padding='pre')
        prediction = model.predict_classes(padded_seq, verbose=0)
        next_word = index_word.get(prediction[0])
        input_words += " " + next_word
    return input_words

In [8]:
for i, raw_text in enumerate([CNN, USAToday, TheGreenMile, HarryPotter]):
    source_text = sent_tokenize(raw_text)
    simple_small = []
    simple_big = []
    complex_small = []
    complex_big = []
    for sent in tqdm_notebook(source_text):        
        result = generate_text(sent, 100, simple_small_model, tokenizer_simple, max_len_simple)
        simple_small.append(result)
        
        result = generate_text(sent, 100, simple_big_model, tokenizer_simple, max_len_simple)
        simple_big.append(result)
        
        result = generate_text(sent, 100, complex_small_model, tokenizer_complex, max_len_complex)
        complex_small.append(result)
        
        result = generate_text(sent, 100, complex_big_model, tokenizer_complex, max_len_complex)
        complex_big.append(result)
    simple_small = ' '.join(simple_small)
    with open('build/generated-texts/{0}-word-based-simple-shallow.txt'.format(source_name[i]), 'w') as f:
        f.write(simple_small)
        
    simple_big = ' '.join(simple_big)
    with open('build/generated-texts/{0}-word-based-simple-deep.txt'.format(source_name[i]), 'w') as f:
        f.write(simple_big)
        
    complex_small = ' '.join(complex_small)
    with open('build/generated-texts/{0}-word-based-complex-shallow.txt'.format(source_name[i]), 'w') as f:
        f.write(complex_small)
        
    complex_big = ' '.join(complex_big)
    with open('build/generated-texts/{0}-word-based-complex-deep.txt'.format(source_name[i]), 'w') as f:
        f.write(complex_big)